In [5]:
import datetime
from sys import displayhook
from pandas_datareader import data as pdr
from datetime import datetime, timedelta

indices = ['^BVSP', '^GSPC']
hoje = datetime.now()
um_ano_atras = hoje - timedelta(days=366)
dado_mercado = pdr.get_data_yahoo(indices, start=um_ano_atras, end=hoje)
displayhook(dado_mercado)

c:\Users\wallison-santos\Documents\GitHub\AppPy\env\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: '^BVSP', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


Attributes    Adj Close              Close               High        \
Symbols           ^GSPC ^BVSP        ^GSPC ^BVSP        ^GSPC ^BVSP   
Date                                                                  
2021-10-19  4519.629883   NaN  4519.629883   NaN  4520.399902   NaN   
2021-10-20  4536.189941   NaN  4536.189941   NaN  4540.870117   NaN   
2021-10-21  4549.779785   NaN  4549.779785   NaN  4551.439941   NaN   
2021-10-22  4544.899902   NaN  4544.899902   NaN  4559.669922   NaN   
2021-10-25  4566.479980   NaN  4566.479980   NaN  4572.620117   NaN   
...                 ...   ...          ...   ...          ...   ...   
2022-10-13  3669.909912   NaN  3669.909912   NaN  3685.409912   NaN   
2022-10-14  3583.070068   NaN  3583.070068   NaN  3712.000000   NaN   
2022-10-17  3677.949951   NaN  3677.949951   NaN  3689.729980   NaN   
2022-10-18  3719.979980   NaN  3719.979980   NaN  3762.790039   NaN   
2022-10-19  3695.159912   NaN  3695.159912   NaN  3728.580078   NaN   

Attributes          Low               Open              Volume        
Symbols           ^GSPC ^BVSP        ^GSPC ^BVSP         ^GSPC ^BVSP  
Date                                                                  
2021-10-19  4496.410156   NaN  4497.339844   NaN  2.531210e+09   NaN  
2021-10-20  4524.399902   NaN  4524.419922   NaN  2.671560e+09   NaN  
2021-10-21  4526.890137   NaN  4532.240234   NaN  3.016950e+09   NaN  
2021-10-22  4524.000000   NaN  4546.120117   NaN  3.062810e+09   NaN  
2021-10-25  4537.359863   NaN  4553.689941   NaN  3.250210e+09   NaN  
...                 ...   ...          ...   ...           ...   ...  
2022-10-13  3491.580078   NaN  3520.370117   NaN  5.021680e+09   NaN  
2022-10-14  3579.679932   NaN  3690.409912   NaN  4.243030e+09   NaN  
2022-10-17  3638.649902   NaN  3638.649902   NaN  4.352780e+09   NaN  
2022-10-18  3686.530029   NaN  3746.260010   NaN  4.483740e+09   NaN  
2022-10-19  3666.510010   NaN  3703.110107   NaN  4.223800e+09   NaN  

[253 rows x 12 columns]